In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%pylab inline
import seaborn as sns

base_path = 'F:\Study\weiboPredict\data'

Populating the interactive namespace from numpy and matplotlib


In [5]:
from scipy.ndimage.interpolation import shift

def extract_features_tt(features_t,repo,k):
    arrive_tt = repo.arrive_tt[repo['w_id']==features_t.name].values
    interval_tt = arrive_tt - shift(arrive_tt,1,cval=0)
    features_t[5:]=arrive_tt
    features_t['max_interval_tt']=interval_tt.max()
    features_t['mean_interval_tt']=interval_tt.mean()
    features_t['mean_interval_fhk_tt'] = interval_tt[:math.ceil(k/2)].mean() # mean of first half k interval
    features_t['mean_interval_lhk_tt'] = interval_tt[math.ceil(k/2):].mean() # mean of last half k interval
    return  features_t

def extract_features_wt(features_t,repo,origin,k):
    arrive_wt = repo.arrive_wt[repo['w_id']==features_t.name]
    interval_wt = arrive_wt - shift(arrive_wt,1,cval=0)
    features_t[5:]=arrive_wt
    post_time = origin_xx.loc[features_t.name].time
    features_t['post_wt'] = post_time.hour*3600 + post_time.minute*60 + post_time.second
    features_t['max_interval_wt']=interval_wt.max()
    features_t['mean_interval_wt']=interval_wt.mean()
    features_t['mean_interval_fhk_wt'] = interval_wt[:math.ceil(k/2)].mean() # mean of first half k interval
    features_t['mean_interval_lhk_wt'] = interval_wt[math.ceil(k/2):].mean() # mean of last half k interval
    return  features_t

In [7]:
repo_str = ['repo_vali','repo_test','repo_train']
origin_str = ['origin_vali','origin_test','origin_train']

for i in range(3):
    origin_xx = pd.read_csv(base_path+'\\samples\\'+origin_str[i]+'_tt.csv',
                     sep='\001',
                     encoding='utf-8',
                     index_col='w_id',
                     quoting=3)
    repo_xx = pd.read_csv(base_path+'\\samples\\'+repo_str[i]+'_k75_tt.csv',
                    sep='\001',
                    encoding='utf-8',
                    quoting=3).fillna(value='', axis=1)
    print('origin: ', len(origin_xx), '\n', 'repo: ', len(repo_xx))
    # 只取datetime中的time
    origin_xx.time = [t.time() for t in pd.to_datetime(origin_xx.time)]
    k = 75
    tt_columns = ['post_tt','max_interval_tt','mean_interval_tt','mean_interval_fhk_tt','mean_interval_lhk_tt']+['arrive_tt_' + str(i) for i in range(k)]
    features_t = pd.DataFrame(index=origin_xx.index, columns=tt_columns)
    features_t = features_t.apply(extract_features_tt,axis=1,repo=repo_xx,k=k)
    features_t['post_tt'] = origin_xx.post_tt
    features_t.to_csv(base_path + '\\features\\features_t_'+repo_str[i].split('_')[1]+'_tt.csv',
                        index_label='w_id',
                        sep='\001',
                        encoding='utf-8',
                        quoting=3)
    
    wt_columns = ['post_wt','max_interval_wt','mean_interval_wt','mean_interval_fhk_wt','mean_interval_lhk_wt']+['arrive_wt_' + str(i) for i in range(k)]
    features_t = pd.DataFrame(index=origin_xx.index, columns=wt_columns)
    features_t = features_t.apply(extract_features_wt,axis=1,repo=repo_xx,origin=origin_xx,k=k)
     
    features_t.to_csv(base_path + '\\features\\features_t_'+repo_str[i].split('_')[1]+'_wt.csv',
                        index_label='w_id',
                        sep='\001',
                        encoding='utf-8',
                        quoting=3)

origin:  4160 
 repo:  312000


origin:  5199 
 repo:  389925


origin:  16640 
 repo:  1248000
